In [ ]:
# https://www.youtube.com/watch?v=QdA-CRr_oXo
# https://learnbybuilding.ai/tutorials/a-gentle-introduction-to-dspy

In [ ]:
!pip install -q dspy-ai lamini rich

In [ ]:
import sys
import os
import dspy
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate
from rich import print
from dsp import LM
import lamini
from dspy.datasets import HotPotQA
import dspy
from dspy.evaluate.evaluate import Evaluate
# from dspy.teleprompt import bootstrapFewShot

In [ ]:
import dspy
import time
from lamini import Lamini

class CustomLMClient(dspy.LM):
    def __init__(self):
        self.provider = "lamini"
        self.history = []
        lamini.api_key = "030f9952a4ee130a33d4fcfd7a7f3dbeab6b2a24a46917b6681019267130e468"
        self.llm = Lamini("meta-llama/Meta-Llama-3.1-8B-Instruct")
        # Add a kwargs attribute to store keyword arguments
        self.kwargs = {'temperature': 0.95, 'max_tokens': 256}

    def basic_request(self, prompt, **kwargs):
        # Call the lamini API to generate a response
        print("$ API Request sent")
        response = self.llm.generate(prompt)
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        # Add the prompt to the history
        self.history.append(prompt)

        # Get the response using the basic_request method
        response = self.basic_request(prompt)

        # Return the response
        return response

    def completions(self):
        return self

    def create(self, messages):
        # Convert the messages to a single prompt
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        response = self.basic_request(prompt)
        # Format the response to match the expected structure
        return type('obj', (object,), {
            'choices': [
                type('obj', (object,), {
                    'message': type('obj', (object,), {
                        'content': response
                    })
                })
            ]
        })

# Create an instance of CustomLMClient
lm = CustomLMClient()

# Configure dspy to use your custom LLM
dspy.settings.configure(lm=lm)

In [ ]:
import requests
from bs4 import BeautifulSoup
import dspy
res = requests.get("https://grugbrain.dev/")
soup = BeautifulSoup(res.text, 'html.parser')
raw_text = [p.text for p in soup.find_all('p') if p.text]

In [ ]:
raw_text[:10]

['this collection of thoughts on software development gathered by grug brain developer',
 'grug brain developer not so smart, but grug brain developer program many long year and learn some things\nalthough mostly still confused',
 'grug brain developer try collect learns into small, easily digestible and funny page, not only for you, the young grug, but also for him\nbecause as grug brain developer get older he forget important things, like what had for breakfast or if put pants on',
 'big brained developers are many, and some not expected to like this, make sour face',
 'THINK they are big brained developers many, many more, and more even definitely probably maybe not like this, many\nsour face (such is internet)',
 '(note: grug once think big brained but learn hard way)',
 'is fine!',
 'is free country sort of and end of day not really matter too much, but grug hope you fun reading and maybe learn from\nmany, many mistake grug make over long program life',
 'apex predator of grug is 

In [ ]:
# from openai import OpenAI
# client = OpenAI()
# openai_model_name= "gpt-3.5-turbo"

class BuildMessages:
    def __init__(self, system_prompt, user_prompt):
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
    def render(self, **kwargs):
        sys = self.system_prompt.format(**kwargs)
        user = self.user_prompt.format(**kwargs)
        return [
            {"role":"system", "content":sys},
            {"role":"user", "content":user},
        ]

from functools import cache

@cache
def translate_grug(grug_text):
    prompt = BuildMessages(
        "You are an expert in deciphering strange text. The user will provide text written by someone named Grug and you will provide the translation.",
        """Translate the following text into plain english: '{text}'.

        Do not respond with any other text. Only provide that text. Now take a deep breath and begin."""
    )
    result = lm.completions().create(messages=prompt.render(text=grug_text))
    return result.choices[0].message.content

In [ ]:
dataset = []
for grug_text in raw_text[:10]:
    translated = translate_grug(grug_text)
    dataset.append({"grug_text":grug_text, "plain_english":translated})

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

$ API Request sent

In [ ]:
examples = []
for row in dataset:
    examples.append(dspy.Example(grug_text=row["grug_text"], plain_english=row["plain_english"]).with_inputs("plain_english"))

In [ ]:
print(examples[0])

Example({'grug_text': 'this collection of thoughts on software development gathered by grug brain developer', 
'plain_english': " \n        'grug think much about code. grug write code. grug make code work. grug happy. grug 
want to share code with others. grug write down thoughts on code. grug hope others learn from grug.' \n        
'grug think about design patterns. grug think about object oriented programming. grug think about testing. grug 
think about refactoring. grug think about many things.' \n        'grug want to write book about software 
development. grug want to share knowledge with others. grug want to help others learn. grug want to make others 
happy.' \n        'grug think about many things. grug think about code. grug think about design patterns. grug 
think about testing. grug think about refactoring. grug think about many things.' \n        'grug want to write 
book about software development. grug want to share knowledge with others. grug want to help others learn. grug 
want to make others happy.' \n        'grug think much about code. grug write code. grug make code work. grug 
happy. grug want to share code with others. grug write down thoughts on code. grug hope others learn from grug.' \n
'grug think about design patterns. grug think about object oriented programming. grug think about testing. grug 
think about refactoring. grug think about many things.' \n        'grug want to write book about software 
development. grug want to share knowledge with others. grug want to help others learn. grug want to make others 
happy.' \n        'grug think much about code. grug write code. grug make code work. grug happy. grug want to share
code with others. grug write down thoughts on code. grug hope others learn from grug.' \n        'grug think about 
design patterns"}) (input_keys={'plain_english'})

In [ ]:
import numpy as np
from random import shuffle
def split_for_train_test(values, test_size = 1/3.0):
    shuffle(values)
    train = int(len(values)-test_size*len(values))
    print(train)
    return values[:train], values[train:]
train, test = split_for_train_test(examples)

6

In [ ]:
train[0]


Example({'grug_text': 'this collection of thoughts on software development gathered by grug brain developer', 'plain_english': " \n        'grug think much about code. grug write code. grug make code work. grug happy. grug want to share code with others. grug write down thoughts on code. grug hope others learn from grug.' \n        'grug think about design patterns. grug think about object oriented programming. grug think about testing. grug think about refactoring. grug think about many things.' \n        'grug want to write book about software development. grug want to share knowledge with others. grug want to help others learn. grug want to make others happy.' \n        'grug think about many things. grug think about code. grug think about design patterns. grug think about testing. grug think about refactoring. grug think about many things.' \n        'grug want to write book about software development. grug want to share knowledge with others. grug want to help others learn. grug w

In [ ]:
import dspy
class GrugTranslation(dspy.Signature):
    "Translate plain english to Grug text."
    plain_english = dspy.InputField()
    grug_text = dspy.OutputField()

In [ ]:
# turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=1000)
# dspy.settings.configure(lm=turbo)

from dspy.signatures.signature import signature_to_template
grug_translation_as_template = signature_to_template(GrugTranslation)
print(str(grug_translation_as_template))

print(grug_translation_as_template.query(examples[0]))

GrugTranslation.signature
GrugTranslation.with_instructions

Template(Translate plain english to Grug text., ['Plain English:', 'Grug Text:'])

Plain English: 'grug think much about code. grug write code. grug make code work. grug happy. grug want to share 
code with others. grug write down thoughts on code. grug hope others learn from grug.' 'grug think about design 
patterns. grug think about object oriented programming. grug think about testing. grug think about refactoring. 
grug think about many things.' 'grug want to write book about software development. grug want to share knowledge 
with others. grug want to help others learn. grug want to make others happy.' 'grug think about many things. grug 
think about code. grug think about design patterns. grug think about testing. grug think about refactoring. grug 
think about many things.' 'grug want to write book about software development. grug want to share knowledge with 
others. grug want to help others learn. grug want to make others happy.' 'grug think much about code. grug write 
code. grug make code work. grug happy. grug want to share code with others. grug write down thoughts on code. grug 
hope others learn from grug.' 'grug think about design patterns. grug think about object oriented programming. grug
think about testing. grug think about refactoring. grug think about many things.' 'grug want to write book about 
software development. grug want to share knowledge with others. grug want to help others learn. grug want to make 
others happy.' 'grug think much about code. grug write code. grug make code work. grug happy. grug want to share 
code with others. grug write down thoughts on code. grug hope others learn from grug.' 'grug think about design 
patterns
Grug Text: this collection of thoughts on software development gathered by grug brain developer

<bound method SignatureMeta.with_instructions of GrugTranslation(plain_english -> grug_text
    instructions='Translate plain english to Grug text.'
    plain_english = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Plain English:', 'desc': '${plain_english}'})
    grug_text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Grug Text:', 'desc': '${grug_text}'})
)>

In [ ]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(GrugTranslation)

    def forward(self, plain_english):
        return self.prog(plain_english=plain_english)
c = CoT()

In [ ]:
c.forward("You should not construct complex systems.")

$ API Request sent

$ API Request sent

Prediction(
    rationale='p',
    grug_text='',
    completions=Completions(...)
) (2186 completions omitted)